# CHEM 546 Homework 1: Simple MD Program

**Author: Benjamin R. Gilbert**

This Python notebook can be used to simulate a Lennard-Jones fluid in the NVE-ensemble with periodic boundary conditions. The particle positions are intialized along a lattice and the initial velocities are sampled from a random distribution and then scaled to match the target temperature. The minimum-image convention is used when evaluating the pairwise interactions and the Verlet algorithm is used for the time-integration. The code is based on the pseudocode presented in chapters 3 and 4 of Frenkel and Smit.

You can run all of the code by selecting "Cell" -> "Run all". Using the default conditions, the code takes approximately 2 minutes on a Intel i7-6700K @ 4.00Ghz.

**Homework Instructions:**

1. Read chapters 3 and 4 of Frenkel and Smit.
2. Run the code in this Python notebook and answer the questions, be sure to include the requested plots. The most helpful sections of Frenkel and Smit are 3.2.2 and 4.1-4.3.

**Library Imports and Notebook Setup**

In [12]:
import numpy as np
import math as mt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors
#import matplotlib.animation as animation
#%matplotlib notebook

In [5]:
# use LaTeX fonts in the plot
#plt.rc('text', usetex=True)
#plt.rc('font', family='serif')
small_ticklabelsize=14
large_ticklabelsize=18
titlesize=18
fig_titlesize=26
axes_labelsize=16
subplot_labelsize=18
colorbar_labelsize=16

fig_dir="./fig_MD_NVE/" # directory for saving figures

# PROGRAM SUBROUTINES

**Questions:**

1. What procedures are performed during the initialization?
2. Write the form of a Lennard-Jones potential and the force on particle $i$ at position $\mathbf{r}_i$ when interacting with particle $j$ at position $\mathbf{r}_j$ through a Lennard-Jones potential (hint: use the dimensionless form with reduced units, *see section 3.2 of Frenkel and Smit*).
3. We use periodic boundary conditions and the minimum image convention for the force evaluation, how does this affect the calculation of the pairwise interactions?
4. Why do we use the Verlet algorithm rather than something like a 4th-order Runge-Kutta scheme to integrate the equations of motion?

**Answers**:

1. Here, we: 

    a. Initialize a grid with length $\sqrt[\text{dim}]{N}$ (where $\text{dim}\leq 3$)

    b. Populate the grid with $N$ particles that are spaced $\texttt{grid\_spacing}$ apart, with position indices stored in the $\texttt{x}$ array.

    c. Randomly assign velocities to these particles, with the following caveats: 

    - After random initialization of velocities, subtract the mean velocity to ensure that the mean velocity is zero (no net drift or momentum in any direction), 
        
    - Scale the velocities in such a way that the average kinetic energy (assuming $m=1$) is $\text{dim}/2 \cdot T$, where $k_B = 1$. Here, $\texttt{temp} = T$. The $\text{dim}$ factor comes from equipartition. 
    This matches the intialized random velocities to match with the initial temperature $\texttt{temp}$.
    
    d. Calculate previous position $x-v \text{d} t$ to setup the Verlet algorithm

2. It seems to me that we disregard forces on any other but one direction. The line $\texttt{ff=48*r2i*r6i*(r6i-0.5)}$ in the $\texttt{force}$ function means that:
$$\begin{align*}
f_x(r) &= \frac{48x}{r^{14}} - \frac{24}{r^8} = -\frac{\partial V}{\partial r} \frac{\partial r}{\partial x} \\
\implies U(r) &= -\int \frac{r}{x}\left(\frac{48x}{r^{14}} - \frac{24x}{r^8}\right)\cdot \text{dd}r \\ 
&= 4\left(\frac{1}{r^{12}} - \frac{1}{r^6}\right) - U_{\text{cutoff}}
\end{align*}$$
which agrees with the line $\texttt{en+=4*r6i*(r6i-1)-ec}$. We ensured that at $r=r_{\text{cutoff}}$, $U(r) = 0$. That being said, shouldn't the condition be $\texttt{r2 < rc**2}$, and not $\texttt{r2 < rc}$?

3. Using PBC with the minimum image convention ensures that distances are wrapped back into the simulation box, as if we are simulating interactions in an infinite lattice. This is a good approximation if we are simulating interactions in bulk media where the particle number is great so surface or boundary effects become negligible.  

4. What makes Verlet algorithm good for MD simulations are because:

- It is a pseudo-symplectic integrator i.e., it will not violate energy conservation as drastically as non-symplectic integrators such as RK4. 
- Verlet is time-reversible, while RK4 is not. This means that if we use RK4 and run the simulation in reverse, we can't obtain the original positions unlike Verlet (limited by machine precision of course)


    

In [6]:
def initialize(dim,N,temp,L,dt):
    
    #x=np.random.uniform(low=0.0,high=L,size=(dim,N))
    #x=np.asfortranarray(x,dtype=np.double)
    
    grid_size=int(mt.ceil(mt.pow(N,1.0/dim)))
    
    grid_spacing=L/grid_size
    
    x=np.zeros((dim,N),dtype=np.double,order='F')
    
    if dim==1:
        print('dim=1')
    elif dim==2:
        count=0
        for i in range(grid_size):
            for j in range(grid_size):
                if count<N:
                    x[:,count]=grid_spacing*np.array([j,i],np.double)+grid_spacing/2.0
                    count+=1
    elif dim==3:
        count=0
        for i in range(grid_size):
            for j in range(grid_size):
                for k in range(grid_size):
                    if count<N:
                        x[:,count]=grid_spacing*np.array([k,j,i],np.double)+grid_spacing/2.0
                        count+=1
        
    
    v=np.random.uniform(low=-0.5,high=0.5,size=(dim,N))
    
    v=np.asfortranarray(v,dtype=np.double)
    
    sumv=np.mean(v,axis=1)
    
    sumv2=np.mean(np.sum(np.power(v,2),axis=0))
    
    fs=mt.sqrt(dim*temp/sumv2)
    
    v=(v-sumv[:,np.newaxis])*fs
    
    xm=x-v*dt
    
    return x, v, xm

def force(dim,N,x,L,rc,ec):
    
    en=0.0
    
    iL=1.0/L
    
    f=np.zeros((dim,N),dtype=np.double,order='F')
    
    for i in range(N-1):
        for j in range(i+1,N):
            
            xr=x[:,i]-x[:,j]
            
            xr-=L*np.rint(xr*iL)
            
            r2=np.sum(np.power(xr,2))
            
            if r2 < rc: #if r-squared is less than cutoff-length rc? Why not rc2?
                
                r2i=1/r2
                r6i=mt.pow(r2i,3)
                
                ff=48*r2i*r6i*(r6i-0.5) #only in the x-direction?
                
                f[:,i]+=ff*xr
                f[:,j]-=ff*xr
            
                en+=4*r6i*(r6i-1)-ec
    
    return f, en

def integrate(dim,N,x,v,xm,f,en,dt):
    
    xx=2*x-xm+f*dt**2
    
    dx=xx-xm
    
    v=dx/(2*dt)
    
    sumv=np.sum(v)
    
    sumv2=np.sum(np.sum(np.power(v,2),axis=0))
    
    xm=x
    x=xx
    
    temp=sumv2/(N*dim)
    
    etot=(en+0.5*sumv2)
    
    return x, xm, v, temp, etot

# SYSTEM SETUP

**Questions:**

1. Why do we use dimensionless units?
2. If we are simulating a system of Argon atoms, what is time unit in seconds?
3. What happens if we increase the size of the time-step?

In [7]:
dt=0.005 # time-step [LJ time units] - 0.005 by default
tf=2000 # final time [number of time-steps] - 2000 by default
sample_rate=20 # sampling rate [number of time-steps] - 20 by default

nt=int(mt.ceil(tf/sample_rate))+1 # max number of steps

dim=2 # number of dimensions
N=100 # number of particles - 100 by default

L=1.0E+2 # system size - 1.0E+2 by default

temp=1.0 # initial temperature - 1.0 by default
rc=2.5 # cutoff length - 2.5 by default

rc2i=1.0/mt.pow(rc,2)
iL=1.0/L

ec=4*(mt.pow(rc2i,6)-mt.pow(rc2i,3)) # cutoff energy


# system initialization
x, v, xm = initialize(dim,N,temp,L,dt)


# allocate arrays to store data
t_arr=np.zeros(nt,dtype=np.double,order='F')
temp_arr=np.zeros(nt,dtype=np.double,order='F')
etot_arr=np.zeros(nt,dtype=np.double,order='F')
v_arr=np.zeros(nt,dtype=np.double,order='F')
k_arr=np.zeros(nt,dtype=np.double,order='F')
x_arr=np.zeros((dim,N,nt),dtype=np.double,order='F')

# write initial values in arrays
f, en = force(dim,N,x,L,rc,ec)
k_arr[0]=np.sum(np.sum(np.power(v,2),axis=0))/2
v_arr[0]=en
etot_arr[0]=v_arr[0]+k_arr[0]
temp_arr[0]=np.sum(np.sum(np.power(v,2),axis=0))/(N*dim)
x_arr[:,:,0]=xm-L*np.rint(xm*iL)+L/2


# set the initial time
t=0.0
i=1
t_old=t

# MAIN LOOP

This will output the sampling times as it runs.

In [8]:
# simulate the system from t until tf
while t < tf:
    
    # calculate forces using minimum image convention and a cutoff of rc
    f, en = force(dim,N,x,L,rc,ec)
    
    # integrate forward in time by dt
    x, xm, v, temp, etot = integrate(dim,N,x,v,xm,f,en,dt)
    
    # advance the time
    t+=1
    
    if t>=t_old+sample_rate or t>=tf:
        
        # print current simulation time
        print('t='+str(t))
        
        # sample time and positions
        t_arr[i]=t
        x_arr[:,:,i]=x-L*np.rint(x*iL)+L/2
        
        # sample energies and temperature
        etot_arr[i]=etot
        v_arr[i]=en
        k_arr[i]=np.sum(np.sum(np.power(v,2),axis=0))/2
        temp_arr[i]=temp
        
        
        ########################################
        # INSERT additional functionality here #
        ########################################
        
        # example - radial distribution function
            
        
        ########################################
        

        # update variables for iteration
        i+=1
        t_old=t

t=20.0
t=40.0
t=60.0
t=80.0
t=100.0
t=120.0
t=140.0
t=160.0
t=180.0
t=200.0
t=220.0
t=240.0
t=260.0
t=280.0
t=300.0
t=320.0
t=340.0
t=360.0
t=380.0
t=400.0
t=420.0
t=440.0
t=460.0
t=480.0
t=500.0
t=520.0
t=540.0
t=560.0
t=580.0
t=600.0
t=620.0
t=640.0
t=660.0
t=680.0
t=700.0
t=720.0
t=740.0
t=760.0
t=780.0
t=800.0
t=820.0
t=840.0
t=860.0
t=880.0
t=900.0
t=920.0
t=940.0
t=960.0
t=980.0
t=1000.0
t=1020.0
t=1040.0
t=1060.0
t=1080.0
t=1100.0
t=1120.0
t=1140.0
t=1160.0
t=1180.0
t=1200.0
t=1220.0
t=1240.0
t=1260.0
t=1280.0
t=1300.0
t=1320.0
t=1340.0
t=1360.0
t=1380.0
t=1400.0
t=1420.0
t=1440.0
t=1460.0
t=1480.0
t=1500.0
t=1520.0
t=1540.0
t=1560.0
t=1580.0
t=1600.0
t=1620.0
t=1640.0
t=1660.0
t=1680.0
t=1700.0
t=1720.0
t=1740.0
t=1760.0
t=1780.0
t=1800.0
t=1820.0
t=1840.0
t=1860.0
t=1880.0
t=1900.0
t=1920.0
t=1940.0
t=1960.0
t=1980.0
t=2000.0


# ANALYSIS

**PLOT TIME-EVOLUTION OF SYSTEM AS IMAGES** - submit the images of the initial and final configurations with your homework

Change the *time_evo_figs* variable to enable this.

Be careful, this makes as many images as you have sample time points and only works for a 2D system. The plot below will be empty, check the figure directory for the series of images. 

In [9]:
time_evo_figs=True # - change this to True to plot the time-evolution images

if dim==2 and time_evo_figs==True:

    fig = plt.figure(figsize=[8, 8])

    ax = plt.axes()

    s=2 # number of periodic images to plot
    
    nt_new=nt
    for i in range(1,nt):
        if t_arr[i]==0.0:
            nt_new=i
            break
    
    L_arr=L*np.ones((dim,N),dtype=np.double,order='F')
    
    for i in range(nt_new):
        
        for j in range(-s,s+1):
            for k in range(-s,s+1):
                shift_arr=L*np.array([j,k],dtype=np.double,order='F')
                
                temp_x=x_arr[:,:,i]+shift_arr[:,np.newaxis]
                
                plt.scatter(temp_x[0,:],temp_x[1,:],marker='o',c=np.arange(0,N,dtype=float),cmap='plasma')
    
        #plt.scatter(x_arr[0,:,i],x_arr[1,:,i],marker='o',c=np.arange(0,N,dtype=np.float),cmap='plasma')
        
        ax.set_xlim(-(s-1)*L,s*L)
        ax.set_ylim(-(s-1)*L,s*L)
        
        
        fig.savefig(fig_dir+"coords_"+str(i)+".png")
        
        plt.cla()
    


<IPython.core.display.Javascript object>

**ENERGY PLOTS** - submit this with your homework

1. Should the total energy stay constant? What about the kinetic and potential energies?
2. Why is there a delay before we start seeing changes in the kinetic and potential energy? How would you change the initial conditions to lengthen or shorten the delay?

**Answers**

1. The KE and PE are generally not conserved because of the forces acting between particles, but ideally, the total mechanical energy should be conserved as we've used a symplectic integrator. 
But such a symplectic integrator only obeys pseudo-Hamiltonian dynamics at discrete, finite time steps $\Delta t$, and the small fluctuations in the mechanical energy should disappear at the limit $\Delta t \to 0$.

2. The delay in the KE and PE fluctuations can be attributed to the integration time step, as it affects the precision in tracking interparticle interactions. We can increase $\Delta t$ to delay the fluctuations, but this leads to higher fluctuations in the mechanical energy and higher deviation from true NVE ensemble. Conversely, we can shorten $\Delta t$ to see the oscillations earlier.


In [ ]:
fig = plt.figure(figsize=[8, 8])
ax = plt.axes()

nt_new=nt
for i in range(1,nt):
    if t_arr[i]==0.0:
        nt_new=i
        break
        

ax.plot(dt*t_arr[:nt_new],k_arr[:nt_new],label=r'Kinetic')
ax.plot(dt*t_arr[:nt_new],v_arr[:nt_new],label=r'Potential')
ax.plot(dt*t_arr[:nt_new],etot_arr[:nt_new],ls='--',label=r'Total')


ax.legend(fontsize=16)

#ax.legend(loc='upper left',fontsize=14)
plt.ylabel(r'Energy',fontsize=axes_labelsize)
plt.xlabel(r'Time',fontsize=axes_labelsize)
plt.show()

fig.savefig(fig_dir+"energy.png")

<IPython.core.display.Javascript object>

**TEMPERATURE PLOT** - submit this with your homework

1. How is the temperature calculated?

**Answer**

1. Note that: 
$$\begin{align*}
r(t+\Delta t) &\approx r(t) + v(t) \Delta t + \frac{f(t)}{2m}(\Delta t)^2 \\
r(t-\Delta t) &\approx r(t) - v(t) \Delta t + \frac{f(t)}{2m}(\Delta t)^2 \\
\implies v(t) &\approx \frac{r(t+\Delta t) - r(t-\Delta t)}{2\Delta t}
\end{align*}$$
Now $r(t+\Delta t)$ and $r(t-\Delta t)$ are stored in the $\texttt{xx}$ and $\texttt{xm}$ arrays in the $\texttt{integrate}$ function. From here, we can then calculate the temperature $T(t)$ as a function of time. In dimensionless units $k_B = m = 1$,
$$\begin{align*}
\frac{\text{dim}}{2}T = \frac{1}{2}\langle v^2 \rangle \implies T &= \frac{\langle v^2 \rangle}{\text{dim}}
\end{align*}$$
where $\langle v^2 \rangle$ is the average of the squared velocities of the particles at a time point.

In [ ]:
fig = plt.figure(figsize=[8, 8])
ax = plt.axes()

nt_new=nt
for i in range(1,nt):
    if t_arr[i]==0.0:
        nt_new=i
        break

ax.plot(dt*t_arr[:nt_new],temp_arr[:nt_new])

    

#ax.legend(loc='upper left',fontsize=14)
plt.ylabel(r'Temperature',fontsize=axes_labelsize)
plt.xlabel(r'Time',fontsize=axes_labelsize)
plt.show()



<IPython.core.display.Javascript object>